# ECE 209 CA3
Zan Xie, UID:205364923<br>
The code runs on Google Colab. Please be aware of file_path if running on different platform.

In [ ]:
# define parse cpp function
from tabulate import tabulate


def parse_cpp(file_path):
  # initialize basci lock list
  block_list = []
  basic_block = []
  inside_main = 0
  line_idx = 1
  inside_loop = 0

  # open cpp file
  with open(file_path, 'r') as file:
    # look at line in loop
    for line in file:
      # detect main section
      if ('int main()' in line):
        inside_main = 1
        continue

      # handle control flows
      if (inside_main):
        line_idx += 1 # current line index

        if ('return' in line):
          # close block before cur_line
          block_list.append(basic_block)
          basic_block = []
          # return itself is a block
          block_list.append([line_idx])
          return block_list

        elif ('break' in line):
          # inclue cur_line and close block
          basic_block.append(line_idx)
          block_list.append(basic_block)
          basic_block = []

        elif('else if' in line or 'else' in line):
          # close block before cur_line
          block_list.append(basic_block)
          basic_block = []
          # next single line as a block if not {
          if ('{' not in line):
            if (';' not in line):
              line_idx += 1
            block_list.append([line_idx])
          else:
            inside_loop += 1

        elif ('if' in line):
          # inclue cur_line and close block
          basic_block.append(line_idx)
          block_list.append(basic_block)
          basic_block = []
          # next single line as a block if not {
          if ('{' not in line):
            if (';' not in line):
              line_idx += 1
            block_list.append([line_idx])
          else:
            inside_loop += 1

        elif ('while' in line or 'for' in line or 'switch' in line):
          inside_loop += 1
          # close block before cur_line
          block_list.append(basic_block)
          basic_block = []
          # while/for/switch itself is a block
          block_list.append([line_idx])

        else:
          basic_block.append(line_idx)
          if (inside_loop > 0 and '}' in line):
            # include cur_line and close block
            block_list.append(basic_block)
            basic_block = []
            inside_loop -= 1

def block_table(block_list):
  tb = []
  for idx, x in enumerate(block_list):
    block = idx+1
    lines = x
    entry_p = x[0]
    exit_p = x[-1]
    if (idx==0):
      entry_p = 1

    data = [block, lines, entry_p, exit_p]
    tb.append(data)

  print(tabulate(tb, headers=["Block", "Lines", "Entry Point", "Exit Point"]))


In [ ]:
# myp.cpp
file_path = '/content/myp.cpp'
block_list = parse_cpp(file_path)
block_table(block_list)

  Block  Lines           Entry Point    Exit Point
-------  ------------  -------------  ------------
      1  [2, 3, 4, 5]              1             5
      2  [6]                       6             6
      3  [7]                       7             7
      4  [8]                       8             8
      5  [9]                       9             9
      6  [10]                     10            10
      7  [11]                     11            11
      8  [12]                     12            12


In [ ]:
# advp.cpp
file_path = '/content/advp.cpp'
block_list = parse_cpp(file_path)
block_table(block_list)

  Block  Lines           Entry Point    Exit Point
-------  ------------  -------------  ------------
      1  [2, 3, 4, 5]              1             5
      2  [6]                       6             6
      3  [8, 9]                    8             9
      4  [10, 11, 12]             10            12
      5  [13]                     13            13
      6  [14, 15]                 14            15
      7  [16]                     16            16
      8  [18, 19]                 18            19
      9  [20]                     20            20
     10  [21, 22, 23]             21            23
     11  [24]                     24            24
     12  [26, 27]                 26            27
     13  [28, 29]                 28            29
     14  [30]                     30            30
     15  [31, 32, 33]             31            33
     16  [34, 35, 36]             34            36
     17  [37, 38, 39]             37            39
     18  [40]                  

#Implementation Logic



Iterate cpp file line by line, and perform certain action if control flow key words found in current_line. Keywords include ['return', 'break', 'if', 'else if', 'else', 'while', 'for', 'switch'].<br>

(1) The function stays inactive until the string 'int main()' found in current_line. line_idx starts increasing by 1 in every iteration.<br>

(2) Append line_idx to basic_block until a keyword is found in current_line. Depending on the type of keyword, either append current_line_index to basic_block or not (INDEX). Append basic_block to block_list as one block unit, erase basic_block.<br>

(3) If control flow keyword found, loop_idx increase by 1, implying that we are inside a control_statement and should keep track of '}' in later lines. Remember to decrease loop_idx by one if '}' found, indicating that control_statement exit.<br>

(4) The keyword 'return' in cpp file is be found eventually, and the function returns with block_list.

#INDEX
What is included in one block.



'return': not include. Itself is a block.<br>

'break': include. Act as an exit point<br>

'if': include. Act as an exit point.<br>

'else if': not include.<br>

'else': not include.<br>

'while': not include. Itself is a block.<br>

'for': not include. Itself is a block.<br>

'switch': not include. Itself is a block.<br>